In [ ]:
import numpy as np


In [ ]:
import pennylane as qml


In [ ]:
n_samples = 10
n_qubits = n_samples  # number of samples tested

#dev_rigetti_aws = qml.device('braket.aws.qubit', device_arn=device_rigetti, shots=10000, wires=n_qubits, s3_destination_folder=s3_folder)
#dev_rigetti_aws = qml.device("default.qubit", wires=n_qubits, shots=10000)
dev_rigetti_aws = qml.device('qiskit.aer', wires=n_qubits)
wires = dev_rigetti_aws.wires.tolist()

@qml.qnode(dev_rigetti_aws)
def q_fc(params, inputs):
    """A variational quantum circuit representing the DRC.

    Args:
        params (array[float]): array of parameters
        inputs (array[float]): array of inputs (data samples)

    Returns:
        array[float]: 1-d output vector in the form of <Z> for every qubit
    """
    
    #print(len(inputs))
    #print(len(params[0]))
    #print(int(len(inputs[0])/3))
    # data sample iteration
    for data in range(len(inputs)):
        # layer iteration
        for l in range(len(params[0])):
            # qubit iteration
            for q in range(1):
                # gate iteration
                for g in range(int(len(inputs[0])/3)):
                  qml.Rot(*(params[0][l][3*g:3*(g+1)] * inputs[data][3*g:3*(g+1)] + params[1][l][3*g:3*(g+1)]), wires=data)

    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]

In [ ]:
num_sample = 5000

X_train = np.loadtxt('X_train_' + str(num_sample) + '.txt')
Y_train = np.loadtxt('Y_train_' + str(num_sample) + '.txt')

X_test = np.loadtxt('X_test_' + str(num_sample) + '.txt')
Y_test = np.loadtxt('Y_test_' + str(num_sample) + '.txt')

# Sanity Check
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(5000, 85) (5000,)
(5000, 85) (5000,)


In [ ]:
important_features = [
    
    'eos_type',
	'v2',
	'v3',
	'v4',
 'v5',
 'psi2',
 'psi3'
	   
]

# Best features according to reference


# The index of the best features in the header_list
important_features_id = np.array([
                         0,
                         2,
                         3,
                         4,
                         5,
                         6,
                         7
                         
])

In [ ]:

X_train = X_train[:, important_features_id[:6]]
X_test = X_test[:, important_features_id[:6]]

X_train.shape, X_test.shape

((5000, 6), (5000, 6))

In [ ]:
best_params = np.array([[[-8.1651270e-01, -5.7219396e+00, -5.0026126e+00, -8.7349653e+00,
          8.6107445e+00, -7.4346608e-01],
        [-4.3116546e-01,  1.9235638e-01,  1.1460174e+01,  1.6732130e+01,
         -7.6773262e+00,  2.3683351e-01],
        [-1.2424114e+00, -1.6424029e+00,  4.3668928e+00,  1.2578176e+01,
          1.3795346e+01,  4.1559759e-01],
        [ 1.5771559e+00, -2.9976025e-01,  4.8244944e+00,  1.2191445e+01,
         -1.2368810e+01,  2.3165625e-01],
        [-4.7297475e-01,  1.0174850e-01,  1.1752117e+00,  4.5879779e+00,
         -4.9621615e+00, -4.0477452e-01],
        [ 7.0455074e-01,  4.7375542e-01,  4.6240917e-01,  6.2393603e+00,
         -8.0264759e+00, -8.2564849e-01 ]],
 
        [[ -1.6269213e-01,  4.1776904e-01, -1.1912645e-01, -1.4228214e+00,
          1.2047335e-01,  2.8740829e-01],
        [ 6.2306011e-01,  1.2985326e+00,  1.0940484e-03,  7.7353078e-01,
          7.6046184e-02,  3.1073821e-01],
        [-7.7920145e-01, -8.7578541e-01, -5.1616389e-01, -9.5997304e-01,
         -1.0806603e+00, -8.3950925e-01],
        [ 8.6785805e-01,  1.0877717e+00,  9.0811890e-01,  5.0817257e-01,
          1.1726918e+00,  4.3069747e-01],
        [ 7.0361930e-01,  3.2143527e-01, -7.2015360e-02, -3.6329076e-01,
          2.7806100e-02,  1.8179013e+00],
        [ 1.1288029e+00,  2.6218903e-01,  6.7197585e-01,  5.4800685e-02,
         -3.0021885e-01, -5.6975257e-01 ]]])

alpha = np.array([0.9943338, 1.0097762])

In [ ]:
# execute the circuit on Rigetti's QPU
# we saved the result for every execution just in case
# Note: please change the directory to save the result accordingly (or just delete/comment it) if you want to run the code

for i in range(100):
    result_0 = q_fc(best_params, X_test[Y_test==0][i*10:(i+1)*10])
    np.savetxt('Testing_label0_sample' + str(i) + '.txt', result_0)
    result_1 = q_fc(best_params, X_test[Y_test==1][i*10:(i+1)*10])
    np.savetxt('Testing_label1_sample' + str(i) + '.txt', result_1)

In [ ]:
z_label_0 = np.zeros((1000,))
z_label_1 = np.zeros((1000,))

# load the results from saved files
for i in range(100):
    temp = np.loadtxt('Testing_label0_sample' + str(i) + '.txt')
    z_label_0[i*10:(i+1)*10] = temp
    
    temp = np.loadtxt('Testing_label1_sample' + str(i) + '.txt')
    z_label_1[i*10:(i+1)*10] = temp

# calculate the α1⟨O1⟩
fidel_1_label_0 = np.clip(alpha[1]*(1 - z_label_0)/2, 0, 1)
fidel_1_label_1 = np.clip(alpha[1]*(1 - z_label_1)/2, 0, 1)

# concatenate all into one array of prediction
Y_pred_1 = np.concatenate((fidel_1_label_0, fidel_1_label_1), axis=0)
Y_label = np.concatenate((np.zeros((1000,), dtype=int), np.ones((1000,), dtype=int)), axis=0)

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(Y_label, Y_pred_1)

0.7829044999999999

In [ ]:
## New 
n_samples = 10
n_qubits = n_samples  # number of samples tested

#dev_rigetti_aws = qml.device('braket.aws.qubit', device_arn=device_rigetti, shots=10000, wires=n_qubits, s3_destination_folder=s3_folder)
#dev_rigetti_aws = qml.device("default.qubit", wires=n_qubits, shots=10000)
dev_rigetti_aws = qml.device('qiskit.basicaer', wires=n_qubits)
wires = dev_rigetti_aws.wires.tolist()

@qml.qnode(dev_rigetti_aws)
def q_fc(params, inputs):
    """A variational quantum circuit representing the DRC.

    Args:
        params (array[float]): array of parameters
        inputs (array[float]): array of inputs (data samples)

    Returns:
        array[float]: 1-d output vector in the form of <Z> for every qubit
    """
    
    #print(len(inputs))
    #print(len(params[0]))
    #print(int(len(inputs[0])/3))
    # data sample iteration
    for data in range(len(inputs)):
        # layer iteration
        for l in range(len(params[0])):
            # qubit iteration
            for q in range(1):
                # gate iteration
                for g in range(int(len(inputs[0])/3)):
                  qml.Rot(*(params[0][l][3*g:3*(g+1)] * inputs[data][3*g:3*(g+1)] + params[1][l][3*g:3*(g+1)]), wires=data)

    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]

In [ ]:
# execute the circuit on Rigetti's QPU
# we saved the result for every execution just in case
# Note: please change the directory to save the result accordingly (or just delete/comment it) if you want to run the code

for i in range(100):
    result_0 = q_fc(best_params, X_test[Y_test==0][i*10:(i+1)*10])
    np.savetxt('Testing_label0_sample' + str(i) + '.txt', result_0)
    result_1 = q_fc(best_params, X_test[Y_test==1][i*10:(i+1)*10])
    np.savetxt('Testing_label1_sample' + str(i) + '.txt', result_1)

In [ ]:
z_label_0 = np.zeros((1000,))
z_label_1 = np.zeros((1000,))

# load the results from saved files
for i in range(100):
    temp = np.loadtxt('Testing_label0_sample' + str(i) + '.txt')
    z_label_0[i*10:(i+1)*10] = temp
    
    temp = np.loadtxt('Testing_label1_sample' + str(i) + '.txt')
    z_label_1[i*10:(i+1)*10] = temp

# calculate the α1⟨O1⟩
fidel_1_label_0 = np.clip(alpha[1]*(1 - z_label_0)/2, 0, 1)
fidel_1_label_1 = np.clip(alpha[1]*(1 - z_label_1)/2, 0, 1)

# concatenate all into one array of prediction
Y_pred_1 = np.concatenate((fidel_1_label_0, fidel_1_label_1), axis=0)
Y_label = np.concatenate((np.zeros((1000,), dtype=int), np.ones((1000,), dtype=int)), axis=0)

In [ ]:
roc_auc_score(Y_label, Y_pred_1)

0.780503

In [ ]:
## another one
## New 
n_samples = 10
n_qubits = n_samples  # number of samples tested

#dev_rigetti_aws = qml.device('braket.aws.qubit', device_arn=device_rigetti, shots=10000, wires=n_qubits, s3_destination_folder=s3_folder)
#dev_rigetti_aws = qml.device("default.qubit", wires=n_qubits, shots=10000)
dev_rigetti_aws =  qml.device('qiskit.ibmq', wires=n_qubits, backend='ibmq_qasm_simulator', ibmqx_token="6834a85b6eef4500c39371983dc17feec0ec055e763d50ad7156d5b291d05e8a56cc6c2e09e5ef2e7608c58a04a700addcd6a1fe9c9d3dbcea7783a7222fdc38")

wires = dev_rigetti_aws.wires.tolist()

@qml.qnode(dev_rigetti_aws)
def q_fc(params, inputs):
    """A variational quantum circuit representing the DRC.

    Args:
        params (array[float]): array of parameters
        inputs (array[float]): array of inputs (data samples)

    Returns:
        array[float]: 1-d output vector in the form of <Z> for every qubit
    """
    
    #print(len(inputs))
    #print(len(params[0]))
    #print(int(len(inputs[0])/3))
    # data sample iteration
    for data in range(len(inputs)):
        # layer iteration
        for l in range(len(params[0])):
            # qubit iteration
            for q in range(1):
                # gate iteration
                for g in range(int(len(inputs[0])/3)):
                  qml.Rot(*(params[0][l][3*g:3*(g+1)] * inputs[data][3*g:3*(g+1)] + params[1][l][3*g:3*(g+1)]), wires=data)

    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]

In [ ]:
# execute the circuit on IBM's QPU
# we saved the result for every execution just in case
# Note: please change the directory to save the result accordingly (or just delete/comment it) if you want to run the code

for i in range(100):
    result_0 = q_fc(best_params, X_test[Y_test==0][i*10:(i+1)*10])
    np.savetxt('Testing_label0_sample' + str(i) + '.txt', result_0)
    result_1 = q_fc(best_params, X_test[Y_test==1][i*10:(i+1)*10])
    np.savetxt('Testing_label1_sample' + str(i) + '.txt', result_1)

In [ ]:
z_label_0 = np.zeros((1000,))
z_label_1 = np.zeros((1000,))

# load the results from saved files
for i in range(100):
    temp = np.loadtxt('Testing_label0_sample' + str(i) + '.txt')
    z_label_0[i*10:(i+1)*10] = temp
    
    temp = np.loadtxt('Testing_label1_sample' + str(i) + '.txt')
    z_label_1[i*10:(i+1)*10] = temp

# calculate the α1⟨O1⟩
fidel_1_label_0 = np.clip(alpha[1]*(1 - z_label_0)/2, 0, 1)
fidel_1_label_1 = np.clip(alpha[1]*(1 - z_label_1)/2, 0, 1)

# concatenate all into one array of prediction
Y_pred_1 = np.concatenate((fidel_1_label_0, fidel_1_label_1), axis=0)
Y_label = np.concatenate((np.zeros((1000,), dtype=int), np.ones((1000,), dtype=int)), axis=0)

In [ ]:
roc_auc_score(Y_label, Y_pred_1)

0.7801585

In [ ]:
## another simulator

In [ ]:
## another one
## New 
n_samples = 5
n_qubits = n_samples  # number of samples tested

#dev_rigetti_aws = qml.device('braket.aws.qubit', device_arn=device_rigetti, shots=10000, wires=n_qubits, s3_destination_folder=s3_folder)
#dev_rigetti_aws = qml.device("default.qubit", wires=n_qubits, shots=10000)
dev_rigetti_aws =  qml.device('qiskit.ibmq', wires=n_qubits, backend='ibmq_manila', ibmqx_token="6834a85b6eef4500c39371983dc17feec0ec055e763d50ad7156d5b291d05e8a56cc6c2e09e5ef2e7608c58a04a700addcd6a1fe9c9d3dbcea7783a7222fdc38")

wires = dev_rigetti_aws.wires.tolist()

@qml.qnode(dev_rigetti_aws)
def q_fc(params, inputs):
    """A variational quantum circuit representing the DRC.

    Args:
        params (array[float]): array of parameters
        inputs (array[float]): array of inputs (data samples)

    Returns:
        array[float]: 1-d output vector in the form of <Z> for every qubit
    """
    
    #print(len(inputs))
    #print(len(params[0]))
    #print(int(len(inputs[0])/3))
    # data sample iteration
    for data in range(len(inputs)):
        # layer iteration
        for l in range(len(params[0])):
            # qubit iteration
            for q in range(1):
                # gate iteration
                for g in range(int(len(inputs[0])/3)):
                  qml.Rot(*(params[0][l][3*g:3*(g+1)] * inputs[data][3*g:3*(g+1)] + params[1][l][3*g:3*(g+1)]), wires=data)

    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]

In [ ]:
# execute the circuit on Rigetti's QPU
# we saved the result for every execution just in case
# Note: please change the directory to save the result accordingly (or just delete/comment it) if you want to run the code

for i in range(100):
    result_0 = q_fc(best_params, X_test[Y_test==0][i*10:(i+1)*10])
    np.savetxt('Testing_label0_sample' + str(i) + '.txt', result_0)
    result_1 = q_fc(best_params, X_test[Y_test==1][i*10:(i+1)*10])
    np.savetxt('Testing_label1_sample' + str(i) + '.txt', result_1)

In [ ]:
z_label_0 = np.zeros((1000,))
z_label_1 = np.zeros((1000,))

# load the results from saved files
for i in range(100):
    temp = np.loadtxt('Testing_label0_sample' + str(i) + '.txt')
    z_label_0[i*10:(i+1)*10] = temp
    
    temp = np.loadtxt('Testing_label1_sample' + str(i) + '.txt')
    z_label_1[i*10:(i+1)*10] = temp

# calculate the α1⟨O1⟩
fidel_1_label_0 = np.clip(alpha[1]*(1 - z_label_0)/2, 0, 1)
fidel_1_label_1 = np.clip(alpha[1]*(1 - z_label_1)/2, 0, 1)

# concatenate all into one array of prediction
Y_pred_1 = np.concatenate((fidel_1_label_0, fidel_1_label_1), axis=0)
Y_label = np.concatenate((np.zeros((1000,), dtype=int), np.ones((1000,), dtype=int)), axis=0)

In [ ]:
roc_auc_score(Y_label, Y_pred_1)